In [1]:
import mhcflurry, seaborn, numpy, pandas, pickle, sklearn, collections, scipy, time
from mhcflurry.dataset import Dataset
import fancyimpute, locale

import sklearn.metrics
import sklearn.cross_validation



Using Theano backend.
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


Couldn't import dot_parser, loading of dot files will not be possible.


In [5]:
min_peptides_to_consider_allele = 10
max_ic50 = 50000

In [6]:
# load training data
all_train_data_with_small_alleles = Dataset.from_csv("../data/bdata.2009.mhci.public.1.txt")
print("Unfiltered training data: %s" % (all_train_data_with_small_alleles,))

# filter out smaller alleles
all_train_data = all_train_data_with_small_alleles.filter_alleles_by_count(min_peptides_to_consider_allele)

print("Training data after filtering: %s" % (all_train_data,))

alleles = all_train_data.unique_alleles()

all_validation_data_with_small_alleles = Dataset.from_csv("../data/bdata.2013.mhci.public.blind.1.txt")

print("Validation data: %s" % (all_validation_data_with_small_alleles,))

all_validation_data = all_validation_data_with_small_alleles.filter_alleles_by_count(min_peptides_to_consider_allele)

print("Validation data after filtering: %s" % (
    all_validation_data,))

Unfiltered training data: Dataset(n=137654, alleles=['ELA-A1', 'Gogo-B0101', 'H-2-DB', 'H-2-DD', 'H-2-KB', 'H-2-KD', 'H-2-KK', 'H-2-LD', 'HLA-A0101', 'HLA-A0201', 'HLA-A0202', 'HLA-A0203', 'HLA-A0205', 'HLA-A0206', 'HLA-A0207', 'HLA-A0210', 'HLA-A0211', 'HLA-A0212', 'HLA-A0216', 'HLA-A0219', 'HLA-A0250', 'HLA-A0301', 'HLA-A0302', 'HLA-A11', 'HLA-A1101', 'HLA-A2', 'HLA-A2301', 'HLA-A2402', 'HLA-A2403', 'HLA-A2501', 'HLA-A26', 'HLA-A2601', 'HLA-A2602', 'HLA-A2603', 'HLA-A2902', 'HLA-A3001', 'HLA-A3002', 'HLA-A3101', 'HLA-A3201', 'HLA-A3301', 'HLA-A6601', 'HLA-A6801', 'HLA-A6802', 'HLA-A6901', 'HLA-A8001', 'HLA-B0702', 'HLA-B0801', 'HLA-B0802', 'HLA-B0803', 'HLA-B1402', 'HLA-B1501', 'HLA-B1502', 'HLA-B1503', 'HLA-B1509', 'HLA-B1517', 'HLA-B1801', 'HLA-B2701', 'HLA-B2702', 'HLA-B2703', 'HLA-B2705', 'HLA-B3501', 'HLA-B3503', 'HLA-B3508', 'HLA-B3801', 'HLA-B3901', 'HLA-B4001', 'HLA-B4002', 'HLA-B4201', 'HLA-B44', 'HLA-B4402', 'HLA-B4403', 'HLA-B4501', 'HLA-B4601', 'HLA-B4801', 'HLA-B5101', '

In [12]:
all_validation_data.get_allele("HLA-A0201")

Dataset(n=2126, alleles={'HLA-A0201'})

In [13]:

def thousands(num):
    locale.setlocale(locale.LC_ALL, 'en_US')
    return locale.format("%d", num, grouping=True)

def format_pair(lst1, lst2):
    return ["%s (%s)" % (thousands(a), thousands(b)) for (a,b) in zip(lst1, lst2)]

description_df = pandas.DataFrame(index= ["BD2009", "BLIND"])
#description_df["Alleles"] = [
#    thousands(len(all_train_data_with_small_alleles.unique_alleles())), 
#    thousands(len(all_validation_data_with_small_alleles.unique_alleles()))
#]
description_df["Alleles"] = [
    thousands(len(all_train_data_with_small_alleles.unique_alleles())), 
    thousands(len(all_validation_data_with_small_alleles.unique_alleles()))
]
description_df["Total IC50 Measurements"] = [
    thousands(len(all_train_data_with_small_alleles.unique_allele_peptide_pairs())), 
    thousands(len(all_validation_data_with_small_alleles.unique_allele_peptide_pairs()))
]

print(description_df.to_latex(index_names=False))

\begin{tabular}{lll}
\toprule
{} & Alleles & Total IC50 Measurements \\
\midrule
BD2009 &     106 &                 137,654 \\
BLIND  &      53 &                  27,680 \\
\bottomrule
\end{tabular}



In [22]:
description_df.to_latex?